In [1]:
import os.path as path
from datetime import datetime
from copy import deepcopy, copy
import numpy as np
# import plotter as pltr
# pltr.set_backend(pltr.MatplotlibBackend)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, models, transforms

In [ ]:
# Utility function to plot images
white = pltr.Color(red=255, green=255, blue=255)
big_white_font = pltr.Font(color=white, size=14)

def show_imgs(imgs, titles):
    frame = pltr.Frame(height_px=800, width_px=2000)
    frame.layout(1, len(imgs))  # Show all the images in a single row
    for img, title in zip(imgs, titles):
        chart = frame.create_chart()
        chart.title = title
        chart.title_font = big_white_font
        chart.show_axes = False
        imgplot = pltr.GrayscaleImage(img)
        chart.add(imgplot)
    frame.show()
    
def tensor2img(tensor):
    img = copy(tensor.numpy())
    img = img.transpose((1, 2, 0))
    img = img * stds + means
    img = np.clip(img, 0, 1)
    img = 255 * img
    img = img.astype(np.int)
    return img

def plot_learning_curves(metrics):
    frame = pltr.Frame(height_px=1100, width_px=2000)
    frame.layout(len(metrics.keys()), 1)
    train_color = pltr.Color(red=7, green=87, blue=124)
    val_color = pltr.Color(red=124, green=7, blue=87)
    for metric, values in metrics.items():
        chart = frame.create_chart()
        chart.title = metric.upper()
        chart.title_font = big_white_font
        chart.x_axis = pltr.Axis(label='Epochs', font=pltr.Font(color=white))
        chart.y_axis.font = pltr.Font(color=white)
        chart.legend_location = pltr.LegendLocation.BEST
        
        epochs = [str(i) for i in range(len(values['train']))]
        
        train_metrics = values['train']
        train_line = pltr.Line(categories=epochs, values=train_metrics, legend='Train', color=train_color)
        chart.add(train_line)
        
        val_metrics = values['val']
        val_line = pltr.Line(categories=epochs, values=val_metrics, legend='Val', color=val_color)
        chart.add(val_line)
    frame.show()

## Load Images

### View without any transforms

In [ ]:
train_datadir = '/data/pytorch/bees_ants/train'
traindata = datasets.ImageFolder(train_datadir)
print(traindata[0])
print(len(traindata))

In [ ]:
titles = []
imgs = []
for i in range(5):
    pil_img, label = traindata[i]
    imgs.append(np.array(pil_img))
    titles.append(traindata.classes[label])
show_imgs(imgs, titles)

In [ ]:
titles = []
imgs = []
for i in range(-1, -6, -1):
    pil_img, label = traindata[i]
    imgs.append(np.array(pil_img))
    titles.append(traindata.classes[label])
show_imgs(imgs, titles)

### With transforms

In [4]:
means = [0.485, 0.456, 0.406]
stds = [0.229, 0.224, 0.225]

train_xforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(means, stds)
])

val_xforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(means, stds)
])

train_datadir = '/data/hymenoptera_data/train'
traindata = datasets.ImageFolder(train_datadir, train_xforms)
trainloader = DataLoader(traindata, batch_size=4, shuffle=True)

val_datadir = '/data/hymenoptera_data/val'
valdata = datasets.ImageFolder(val_datadir, val_xforms)
valloader = DataLoader(valdata, batch_size=4, shuffle=True)

In [ ]:
images, labels = next(iter(trainloader))
imgs = [tensor2img(image) for image in images]
titles = [traindata.classes[label] for label in labels]
show_imgs(imgs, titles)

In [ ]:
num = 0
for batch in trainloader:
    num += batch[0].size(0)
num

## Training

In [5]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
def train(model, loss_fn, optimizer, lr, num_epochs=25):
    start = datetime.now()
    metrics = {
        'loss': {
            'val': [],
            'train': []
        },
        'acc': {
            'val': [],
            'train': []
        }
    }
    
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        
        # Training
        model.train()
        lr.step()
        train_loss = 0.0
        train_acc = 0
        
        for images, labels in trainloader:
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)
            with torch.enable_grad():
                optimizer.zero_grad()
                outputs = model(images)
                loss = loss_fn(outputs, labels)
                loss.backward()
                optimizer.step()

                train_loss += loss.item() * images.size(0)
                preds = torch.argmax(outputs, dim=1)
                train_acc += torch.sum(preds == labels.data)
            
        epoch_train_loss = train_loss / len(traindata)
        epoch_train_acc = train_acc.double() / len(traindata)
        metrics['loss']['train'].append(epoch_train_loss)
        metrics['acc']['train'].append(epoch_train_acc.item())
        print(f'Train - Loss: {epoch_train_loss:.3f}\tAcc: {epoch_train_acc:.3f}')
        
        # Validation
        model.eval()
        val_loss = 0.0
        val_acc = 0
        for images, labels in valloader:
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)
            with torch.no_grad():
                outputs = model(images)
                loss = loss_fn(outputs, labels)
                
                preds = torch.argmax(outputs, dim=1)
                val_loss += loss.item() * images.size(0)
                val_acc += torch.sum(preds == labels.data)
        
        epoch_val_loss = val_loss / len(valdata)
        epoch_val_acc = val_acc.double() / len(valdata)
        metrics['loss']['val'].append(epoch_val_loss)
        metrics['acc']['val'].append(epoch_val_acc.item())
        print(f'Val - Loss: {epoch_val_loss:.3f}\tAcc: {epoch_val_acc:.3f}')
    
    end = datetime.now()
    print(f'Training completed in {end-start}')
    return model, metrics

## Fine-Tuning Resnet

In [7]:
model = models.resnet18(pretrained=True)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)
model = model.to(DEVICE)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# Decay learning rate by a factor of 0.1 every 7 epochs
exp_lr = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [9]:
model, metrics = train(model, loss_fn, optimizer, exp_lr, num_epochs=25)


Epoch 1/25


/opt/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:82: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train - Loss: 0.547	Acc: 0.697
Val - Loss: 0.344	Acc: 0.869

Epoch 2/25
Train - Loss: 0.473	Acc: 0.816
Val - Loss: 0.232	Acc: 0.908

Epoch 3/25
Train - Loss: 0.443	Acc: 0.816
Val - Loss: 0.274	Acc: 0.889

Epoch 4/25
Train - Loss: 0.365	Acc: 0.844
Val - Loss: 0.301	Acc: 0.895

Epoch 5/25
Train - Loss: 0.500	Acc: 0.824
Val - Loss: 0.247	Acc: 0.922

Epoch 6/25
Train - Loss: 0.461	Acc: 0.824
Val - Loss: 0.381	Acc: 0.869

Epoch 7/25
Train - Loss: 0.377	Acc: 0.840
Val - Loss: 0.321	Acc: 0.902

Epoch 8/25
Train - Loss: 0.314	Acc: 0.852
Val - Loss: 0.272	Acc: 0.908

Epoch 9/25
Train - Loss: 0.265	Acc: 0.902
Val - Loss: 0.266	Acc: 0.902

Epoch 10/25
Train - Loss: 0.303	Acc: 0.881
Val - Loss: 0.235	Acc: 0.908

Epoch 11/25
Train - Loss: 0.290	Acc: 0.885
Val - Loss: 0.229	Acc: 0.915

Epoch 12/25
Train - Loss: 0.319	Acc: 0.865
Val - Loss: 0.233	Acc: 0.895

Epoch 13/25
Train - Loss: 0.213	Acc: 0.910
Val - Loss: 0.211	Acc: 0.935

Epoch 14/25
Train - Loss: 0.259	Acc: 0.902
Val - Loss: 0.223	Acc: 0.922

In [ ]:
plot_learning_curves(metrics)

In [ ]:
images, labels = next(iter(valloader))
preds = torch.argmax(model(images.to(DEVICE)), dim=1)
imgs = [tensor2img(image) for image in images]
titles = [f'Predicted: {valdata.classes[pred]} Actual: {valdata.classes[label]}' for label, pred in zip(labels, preds)]
show_imgs(imgs, titles)

## Embeddings from Resnet

In [ ]:
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
    
num_featuers = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # This will have requires_grad TRUE
model = model.to(DEVICE)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.01, momentum=0.9)
exp_lr = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
model, metrics = train(model, loss_fn, optimizer, exp_lr, num_epochs=25)

In [ ]:
plot_learning_curves(metrics)

In [ ]:
images, labels = next(iter(valloader))
preds = torch.argmax(model(images.to(DEVICE)), dim=1)
imgs = [tensor2img(image) for image in images]
titles = [f'Predicted: {valdata.classes[pred]} Actual: {valdata.classes[label]}' for label, pred in zip(labels, preds)]
show_imgs(imgs, titles)